In [2]:
#! pip install py2neo;
from py2neo import Graph
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import numpy as np

In [37]:
graph = Graph("bolt://3.220.233.169:7687", auth=("neo4j", "i-0e23d19f0d8795714"))

In [4]:
user_id=195103
biz_id=17

In [20]:
friend_count=graph.run("""MATCH (c:User)
RETURN ID(c) as ident, apoc.node.degree(c, 'FRIEND') as count
""").to_data_frame()

In [21]:
friend_count.describe()

,count,ident
count,1.637138e+06,1.637138e+06
mean,2.754502e-01,3.490227e+06
std,1.131968e+00,2.408194e+06
min,0.000000e+00,1.950860e+05
25%,0.000000e+00,1.344378e+06
50%,0.000000e+00,3.063662e+06
75%,0.000000e+00,5.402947e+06
max,3.450000e+02,8.512231e+06


In [35]:
friend_count['count'].quantile(.80)

0.0

In [25]:
friend_count.loc[friend_count['count']==0].shape

(1312387, 2)

In [38]:
friends=graph.run("""
CALL algo.unionFind.stream('User', 'FRIEND')
YIELD nodeId, setId
""").to_data_frame()

In [52]:
clusters=friends.loc[friends.setId.duplicated(keep=False)]

In [59]:
clusters.setId.value_counts().describe()

count    99316.000000
mean         3.269876
std          5.861368
min          2.000000
25%          2.000000
50%          2.000000
75%          3.000000
max        426.000000
Name: setId, dtype: float64